In [3]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_bayesimp as file

In [16]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.1
niter = 1000
optimise_mu = [True, True]
exact = [False, True]
mc_samples = 100
kernel = "gammaexp"

In [17]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37295 instead
  warnings.warn(
2024-09-22 21:38:14,820 - distributed.batched - INFO - Batched Comm Closed <TCP (closed)  local=tcp://192.168.234.109:36991 remote=tcp://192.168.234.109:41218>
Traceback (most recent call last):
  File "/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/batched.py", line 115, in _background_send
    nbytes = yield coro
             ^^^^^^^^^^
  File "/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/tornado/gen.py", line 766, in run
    value = future.result()
            ^^^^^^^^^^^^^^^
  File "/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/comm/tcp.py", line 262, in write
    raise CommClosedError()
dist

In [18]:
# Submitting jobs
futures = []
for i in range(len(optimise_mu)):           
    for seed in range(ntrial):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                             optimise_mu = optimise_mu[i], exact = exact[i],
                             mc_samples = mc_samples, kernel = kernel,
                             niter = niter)
            futures += [f]

In [5]:
# Check on futures
#futures

In [19]:
# Getting results
results = client.gather(futures)

In [20]:
# Closing client
client.close()
cluster.close()

In [21]:
# Saving results
torch.save(obj = results,
           f = "ablation_bayesimp_ntrial={0}_n={1}_d={2}_noise={3}_kernel={4}.pt".format(ntrial,n,d,noise,kernel))